In [1]:
%load_ext autoreload
%load_ext autotime

time: 173 µs (started: 2024-10-08 21:44:57 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 683 µs (started: 2024-10-08 21:44:57 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 777 µs (started: 2024-10-08 21:44:57 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-montage-pegasus-dss-2deg-node-4"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/montage-pegasus-dss-2deg-node-4"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/montage/pegasus-dss-2deg/node-4/v1/COMPACT/*.pfw.gz"


time: 715 µs (started: 2024-10-08 21:44:57 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 8.71 s (started: 2024-10-08 21:44:57 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 8min 12s (started: 2024-10-08 21:45:06 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          1060.98 seconds                                                                               │
│  I/O Time         2.06 seconds                                                                                  │
│                   ├── Read - 0.05 seconds (2.50%)                                                               │
│                   ├── Write - 0.01 seconds (0.47%)                                                              │
│                   └── Metadata - 2.06 seconds (99.97%)                                                          │
│  I/O Operations   1,674,447 ops                                                                                 │
│                   ├── Read - 79,940 ops (4.77%)                                                                 │
│                   ├── Write - 26,224 ops (1.57%)                                                                │
│                   └── Metadata - 1,566,555 ops (93.56%)                                                         │
│  I/O Size         799.85 MiB                                                                                    │
│                   ├── Read - 5.94 MiB (0.74%)                                                                   │
│                   └── Write - 793.91 MiB (99.26%)                                                               │
│  Read Requests    4 kiB-4 kiB - 73,492 ops                                                                      │
│                   └── <4 kiB - 73,492 ops (100.00%)                                                             │
│  Write Requests   4 kiB-64 kiB - 26,224 ops                                                                     │
│                   ├── <4 kiB - 13,360 ops (50.95%)                                                              │
│                   └── 16-64 kiB - 12,864 ops (49.05%)                                                           │
│  Nodes            4 nodes                                                                                       │
│                   ├── 14275 - 2.06 s (100.00%) - 1.53 MiB/198.50 MiB R/W (0.19/24.82%) - 454,930 ops (27.17%)   │
│                   ├── 29933 - 1.84 s (89.62%) - 1.48 MiB/198.47 MiB R/W (0.19/24.81%) - 406,273 ops (24.26%)    │
│                   ├── 7703 - 1.27 s (61.54%) - 1.48 MiB/198.47 MiB R/W (0.19/24.81%) - 404,826 ops (24.18%)     │
│                   └── 38179 - 1.04 s (50.45%) - 1.45 MiB/198.47 MiB R/W (0.18/24.81%) - 408,418 ops (24.39%)    │
│  Apps             1 app                                                                                         │
│  Processes/Ranks  20,397 processes                                                                              │
│  Files            39,978 files                                                                                  │
│                   ├── Shared: 24,021 files (60.09%)                                                             │
│                   └── FPP: 15,957 files (39.91%)                                                                │
│  Time Periods     385 time periods (Time Granularity: 1,000,000.0)                                              │
│  Access Pattern   Sequential: 106,164 ops (100.00%) - Random: 0 ops (0.00%)                                     │
│                                                                                                                 │
╰─ R: Read - W: Write - M: Metadata  ─────────────────────────────────────────────────────────────────────────────╯
                                                                                                                   
╭───────────────────────────────────── 637 I/O Bottlenec

time: 21.2 s (started: 2024-10-08 21:53:18 -07:00)
